In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import acquire
from env import host, user, password
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

In [2]:
def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def new_telco_data():
    sql_query = 'SELECT customers.*, contract_types.contract_type, internet_service_types.internet_service_type, payment_types.payment_type FROM customers JOIN contract_types using(contract_type_id) JOIN internet_service_types using(internet_service_type_id) JOIN payment_types using(payment_type_id) '
    df = pd.read_sql(sql_query, get_connection('telco_churn'))
    df.to_csv('telco_churn_df.csv')
    return df

In [3]:
acquire

<module 'acquire' from '/Users/matthewknight/telco_project/acquire.py'>

In [4]:
df = new_telco_data()

In [5]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,Yes,1,No,2,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,Yes,1,Yes,4,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,1,Yes,1,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,No,1,Yes,1,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,No,1,No,3,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
senior_citizen,7043.0,0.162147,0.368612,0.00,0.0,0.00,0.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
internet_service_type_id,7043.0,1.872923,0.737796,1.00,1.0,2.00,2.00,3.00
contract_type_id,7043.0,1.690473,0.833755,1.00,1.0,1.00,2.00,3.00
payment_type_id,7043.0,2.315633,1.148907,1.00,1.0,2.00,3.00,4.00
monthly_charges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


In [8]:
telco = df.drop(columns=['customer_id', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'internet_service_type_id', 'contract_type_id', 'payment_type_id'])

In [9]:
telco.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Male,0,No,No,9,Yes,Yes,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,Female,0,No,Yes,9,Yes,No,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,Female,1,No,No,7,Yes,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,Male,1,No,No,1,No,No phone service,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,Female,0,Yes,Yes,1,No,No phone service,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [10]:
# for col in telco.columns:
#     plt.figure(figsize=(4,3))
#     plt.hist(telco[col])
#     plt.title(col)
#     plt.show()

In [11]:
telco = telco.drop(columns=['partner', 'dependents', 'multiple_lines', 'streaming_tv', 'streaming_movies', 'total_charges'])

In [12]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,contract_type,internet_service_type,payment_type
0,Male,0,9,Yes,No,59.9,No,Month-to-month,DSL,Mailed check
1,Female,0,9,Yes,Yes,69.4,No,Month-to-month,DSL,Credit card (automatic)
2,Female,1,7,Yes,Yes,48.2,No,Month-to-month,DSL,Electronic check
3,Male,1,1,No,Yes,25.1,Yes,Month-to-month,DSL,Electronic check
4,Female,0,1,No,No,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [13]:
telco_dummies = pd.get_dummies(telco.contract_type)


In [14]:
telco = pd.concat([telco, telco_dummies], axis=1)


In [15]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,contract_type,internet_service_type,payment_type,Month-to-month,One year,Two year
0,Male,0,9,Yes,No,59.9,No,Month-to-month,DSL,Mailed check,1,0,0
1,Female,0,9,Yes,Yes,69.4,No,Month-to-month,DSL,Credit card (automatic),1,0,0
2,Female,1,7,Yes,Yes,48.2,No,Month-to-month,DSL,Electronic check,1,0,0
3,Male,1,1,No,Yes,25.1,Yes,Month-to-month,DSL,Electronic check,1,0,0
4,Female,0,1,No,No,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),1,0,0


In [16]:
telco = telco.drop(columns='contract_type')

In [17]:
telco_dummies = pd.get_dummies(telco.internet_service_type)


In [18]:
telco = pd.concat([telco, telco_dummies], axis=1)


In [19]:
telco = telco.drop(columns='internet_service_type')

In [20]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,payment_type,Month-to-month,One year,Two year,DSL,Fiber optic,None
0,Male,0,9,Yes,No,59.9,No,Mailed check,1,0,0,1,0,0
1,Female,0,9,Yes,Yes,69.4,No,Credit card (automatic),1,0,0,1,0,0
2,Female,1,7,Yes,Yes,48.2,No,Electronic check,1,0,0,1,0,0
3,Male,1,1,No,Yes,25.1,Yes,Electronic check,1,0,0,1,0,0
4,Female,0,1,No,No,30.5,Yes,Bank transfer (automatic),1,0,0,1,0,0


In [21]:
telco_dummies = pd.get_dummies(telco.payment_type)


In [22]:
telco = pd.concat([telco, telco_dummies], axis=1)


In [23]:
telco = telco.drop(columns='payment_type')

In [24]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,Month-to-month,One year,Two year,DSL,Fiber optic,None,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,Male,0,9,Yes,No,59.9,No,1,0,0,1,0,0,0,0,0,1
1,Female,0,9,Yes,Yes,69.4,No,1,0,0,1,0,0,0,1,0,0
2,Female,1,7,Yes,Yes,48.2,No,1,0,0,1,0,0,0,0,1,0
3,Male,1,1,No,Yes,25.1,Yes,1,0,0,1,0,0,0,0,1,0
4,Female,0,1,No,No,30.5,Yes,1,0,0,1,0,0,1,0,0,0


In [28]:
telco.loc[telco['phone_service'] == 'No', 'phone_service'] = 0

In [29]:
telco.loc[telco['phone_service'] == 'Yes', 'phone_service'] = 1

In [30]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,Month-to-month,One year,Two year,DSL,Fiber optic,None,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,Male,0,9,1,No,59.9,No,1,0,0,1,0,0,0,0,0,1
1,Female,0,9,1,Yes,69.4,No,1,0,0,1,0,0,0,1,0,0
2,Female,1,7,1,Yes,48.2,No,1,0,0,1,0,0,0,0,1,0
3,Male,1,1,0,Yes,25.1,Yes,1,0,0,1,0,0,0,0,1,0
4,Female,0,1,0,No,30.5,Yes,1,0,0,1,0,0,1,0,0,0


In [31]:
telco.loc[telco['paperless_billing'] == 'No', 'paperless_billing'] = 0

In [32]:
telco.loc[telco['paperless_billing'] == 'Yes', 'paperless_billing'] = 1

In [33]:
telco.head()

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,Month-to-month,One year,Two year,DSL,Fiber optic,None,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,Male,0,9,1,0,59.9,No,1,0,0,1,0,0,0,0,0,1
1,Female,0,9,1,1,69.4,No,1,0,0,1,0,0,0,1,0,0
2,Female,1,7,1,1,48.2,No,1,0,0,1,0,0,0,0,1,0
3,Male,1,1,0,1,25.1,Yes,1,0,0,1,0,0,0,0,1,0
4,Female,0,1,0,0,30.5,Yes,1,0,0,1,0,0,1,0,0,0


In [34]:
telco.loc[telco['churn'] == 'No', 'churn'] = 0

In [35]:
telco.loc[telco['churn'] == 'Yes', 'churn'] = 1

In [36]:
telco.head(10)

,gender,senior_citizen,tenure,phone_service,paperless_billing,monthly_charges,churn,Month-to-month,One year,Two year,DSL,Fiber optic,None,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,Male,0,9,1,0,59.90,0,1,0,0,1,0,0,0,0,0,1
1,Female,0,9,1,1,69.40,0,1,0,0,1,0,0,0,1,0,0
2,Female,1,7,1,1,48.20,0,1,0,0,1,0,0,0,0,1,0
3,Male,1,1,0,1,25.10,1,1,0,0,1,0,0,0,0,1,0
4,Female,0,1,0,0,30.50,1,1,0,0,1,0,0,1,0,0,0
5,Male,1,2,1,1,49.25,1,1,0,0,1,0,0,0,0,1,0
6,Male,0,29,0,1,45.00,0,1,0,0,1,0,0,0,0,0,1
7,Male,0,1,1,1,44.30,0,1,0,0,1,0,0,0,0,0,1
8,Female,0,6,1,0,50.35,0,1,0,0,1,0,0,0,0,1,0
9,Male,0,30,1,1,64.50,0,1,0,0,1,0,0,0,0,0,1


In [37]:
telco_dummies = pd.get_dummies(telco.gender)
